## Configuration

In [ ]:
import os
from dotenv import load_dotenv
import requests

load_dotenv()

positions_zip = "positions.zip"
iupds_zip = "iupds.zip"


if os.path.exists(iupds_zip):
    os.remove(iupds_zip)


aca_upload_path="https://api.uat.platform.pagopa.it/upload/aca/debt-positions-service/v2"
aca_path="https://api.uat.platform.pagopa.it/aca/debt-positions-service/v1"

subscription_key = os.getenv('MY_SUBSCRIPTION_KEY')
broker = os.getenv('PAGOPA_TEST_BROKER')
organization = os.getenv('PAGOPA_TEST_ORGANIZATION')

segregation_code = os.getenv('PAGOPA_TEST_SEGREGATION_CODE')

print("Configuration set successfully.")

## Add dynamic values to json template

In [ ]:
import json
import random
import string

file_path = 'PaymentPositions.json'
try:
    with open(file_path, 'r') as file:
        data = json.load(file)

    paymentPositions = data['paymentPositions']
    for paymentPosition in paymentPositions:
        generated_iupd = ''.join(random.choices(string.ascii_uppercase + string.digits, k=15))
        paymentPosition['iupd'] = generated_iupd    
        paymentOptions = paymentPosition['paymentOption']
        for paymentOption in paymentOptions:
            generated_iuv = segregation_code + ''.join(random.choices(string.digits, k=15))
            paymentOption['iuv'] = generated_iuv
            paymentOption['description'] = 'Posizione debitoria ACA test'

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"JSON file '{file_path}' modified successfully.")

except FileNotFoundError:
    print(f"Error: file '{file_path}' was not found.")
except json.JSONDecodeError:
    print(f"Error: file '{file_path}' is not a valid JSON.")

## Create zip from PaymentPositions file

In [ ]:
import zipfile

# Create a zip file containing the JSON file
json_filename = 'PaymentPositions.json'

if os.path.exists(positions_zip):
    os.remove(positions_zip)

with zipfile.ZipFile(positions_zip, 'w') as zipf:
    zipf.write(json_filename, compress_type=zipfile.ZIP_DEFLATED)

print(f'Created {positions_zip} containing {json_filename}')

# CREATE

| Field | Value |
| :--- | :---: |
| Operation | Massive Create |
| Body | Debt Position List |
| URI | `{base-path}/brokers/{broker}/organizations/{organization}/debtpositions/file` |

In [ ]:
url = aca_upload_path+"/brokers/"+broker+"/organizations/"+organization+"/debtpositions/file"

with open('./'+positions_zip, 'rb') as file:
    files = [
        ('file', (positions_zip, file, 'application/zip'))
    ]
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key
    }
    response = requests.request("POST", url, headers=headers, files=files)

    status_location = response.headers['Location']
    print(f"Response status code: {str(response.status_code)}")
    print(f"Status URI: {status_location}")

## Status

| Field | Value |
| :--- | :---: |
| Operation | GET Status |
| Param | UploadKey |
| URI | `{base-path}/brokers/{broker}/organizations/{organization}/debtpositions/file/{uploadKey}/status` |

In [ ]:
url = aca_upload_path + "/" + status_location

response = requests.request("GET", url, headers=headers)

print(response.text)

## Report

| Field | Value |
| :--- | :---: |
| Operation | GET Report |
| Param | UploadKey |
| URI | `{base-path}/brokers/{broker}/organizations/{organization}/debtpositions/file/{uploadKey}/report` |

In [ ]:
report_location = status_location.replace('status', 'report')
url = aca_upload_path + "/" + report_location

response = requests.request("GET", url, headers=headers)

print(response.text)

## Check debt position

In [ ]:
url =aca_path+"/organizations/"+organization+"/debtpositions/" + generated_iupd

response = requests.request("GET", url, headers=headers)

print(response.text)

# UPDATE

| Field | Value |
| :--- | :---: |
| Operation | Massive Update |
| Body | Debt Position List |
| URI | `{base-path}/brokers/{broker}/organizations/{organization}/debtpositions/file` |

## Modify dinamically the payment option description field inside payment position file

In [ ]:
import json

file_path = 'PaymentPositions.json'
try:
    with open(file_path, 'r') as file:
        data = json.load(file)

    paymentPositions = data['paymentPositions']
    for paymentPosition in paymentPositions:
        paymentOptions = paymentPosition['paymentOption']
        for paymentOption in paymentOptions:
            paymentOption['description'] = 'Posizione debitoria ACA test updated'

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"JSON file '{file_path}' modified successfully.")

except FileNotFoundError:
    print(f"Error: file '{file_path}' was not found.")
except json.JSONDecodeError:
    print(f"Error: file '{file_path}' is not a valid JSON.")

## Recreate zip file from updated json

In [ ]:
import zipfile

# Create a zip file containing the JSON file
json_filename = 'PaymentPositions.json'

if os.path.exists(positions_zip):
    os.remove(positions_zip)

with zipfile.ZipFile(positions_zip, 'w') as zipf:
    zipf.write(json_filename, compress_type=zipfile.ZIP_DEFLATED)

print(f"Created {positions_zip} containing {json_filename}")

## Update operation

| Field | Value |
| :--- | :---: |
| Operation | Massive Update |
| Body | Debt Position List |
| URI | `{base-path}/brokers/{broker}/organizations/{organization}/debtpositions/file` |

In [ ]:
url = aca_upload_path+"/brokers/"+broker+"/organizations/"+organization+"/debtpositions/file"

with open('./'+positions_zip, 'rb') as file:
    files = [
        ('file', (positions_zip, file, 'application/zip'))
    ]
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key
    }
    response = requests.request("PUT", url, headers=headers, files=files)

    status_location = response.headers['Location']
    print(f"Response status code: {str(response.status_code)}")
    print(f"Status URI: {str(response.status_code)}")

## Check debt position

In [ ]:
url =aca_path+"/organizations/"+organization+"/debtpositions/" + generated_iupd

response = requests.request("GET", url, headers=headers)

print(response.text)

# DELETE

## Add IUPD to list

In [ ]:
import json

file_path = 'PaymentPositionsIDs.json'
try:
    with open(file_path, 'r') as file:
        data = json.load(file)

    data["paymentPositionIUPDs"] = [generated_iupd]

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"JSON file '{file_path}' modified successfully.")

except FileNotFoundError:
    print(f"Error: file '{file_path}' was not found.")
except json.JSONDecodeError:
    print(f"Error: file '{file_path}' is not a valid JSON.")

## Create zip file from IUPDs json

In [ ]:
import zipfile

# Create a zip file containing the JSON file
json_filename = 'PaymentPositionsIDs.json'

if os.path.exists(iupds_zip):
    os.remove(iupds_zip)

with zipfile.ZipFile(iupds_zip, 'w') as zipf:
    zipf.write(json_filename, compress_type=zipfile.ZIP_DEFLATED)

print(f"Created {iupds_zip} containing {json_filename}")

## Delete operation

| Field | Value |
| :--- | :---: |
| Operation | Massive Delete |
| Body | IUPD List |
| URI | `{base-path}/brokers/{broker}/organizations/{organization}/debtpositions/file` |

In [ ]:
url = aca_upload_path+"/brokers/"+broker+"/organizations/"+organization+"/debtpositions/file"

with open('./'+iupds_zip, 'rb') as file:
    files = [
        ('file', (iupds_zip, file, 'application/zip'))
    ]
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key
    }
    response = requests.request("DELETE", url, headers=headers, files=files)


    status_location = response.headers['Location']
    print(f"Response status code: {str(response.status_code)}")
    print(f"Status URI: {status_location}")

## Status

| Field | Value |
| :--- | :---: |
| Operation | GET Status |
| Param | UploadKey |
| URI | `{base-path}/brokers/{broker}/organizations/{organization}/debtpositions/file/{uploadKey}/status` |

In [ ]:
url = aca_upload_path + "/" + status_location

response = requests.request("GET", url, headers=headers)

print(response.text)

## Report

| Field | Value |
| :--- | :---: |
| Operation | GET Report |
| Param | UploadKey |
| URI | `{base-path}/brokers/{broker}/organizations/{organization}/debtpositions/file/{uploadKey}/report` |

In [ ]:
report_location = status_location.replace('status', 'report')
url = aca_upload_path + "/" + report_location

response = requests.request("GET", url, headers=headers)

print(response.text)

## Check debt position

In [ ]:
url =aca_path+"/organizations/"+organization+"/debtpositions/" + generated_iupd

response = requests.request("GET", url, headers=headers)

print(response.text)